In [51]:
import os
import re
import glob
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
from sklearn.manifold import TSNE
#from cuml.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import MiniBatchKMeans, KMeans, DBSCAN, Birch
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D


In [2]:
def read_csv(prefix):
    path = os.getcwd()
    csv_files = glob.glob(os.path.join(path, "{}*".format(prefix)))
    list_df = []
    for f in csv_files:
        df = pd.read_csv(f)
        list_df.append(df)
        df_c = pd.concat(list_df, ignore_index=True)
    return df_c

In [3]:
def search_candidates(df1,df2):
    dict1 = {}
    for i in range(len(df1['Sequence'])):
        for j in range(len(df2['motifs'])):
            x = df2.iloc[j]['motifs']
            if '*'in x:
                x = x.replace('*','[ATGC]')
            if re.search(x,df1.iloc[i]['Sequence']):
                dict1[df2.iloc[j]['motifs']] = [df2.iloc[j]['index_pos'],df2.iloc[j]['index_neg']]
    dict_new = {}
    for k in sorted(dict1, key=lambda k: len(k), reverse=True):
        dict_new[k] = dict1[k]
    return dict_new

In [4]:
def get_motif_index(dictionary,sequence):
    index_pos = dictionary[sequence][0].strip('[]').split(',')
    index_neg = dictionary[sequence][1].strip('[]').split(',')
    index_pos = [int(x) for x in index_pos]
    index_neg = [int(x) for x in index_neg]
    return index_pos,index_neg

In [8]:
def change_class(df,index_list,n):
    df_n = df.copy(deep=True)
    for index in index_list:
        df_n.loc[index,'Class'] = n
    return df_n


In [9]:
def extract_subdf(df,index_list,n):
    df_new = pd.DataFrame(columns=['Sequence','Class'])
    for i in range(len(index_list)):
        index_n = int(index_list[i])
        df_new.loc[i]=[df.loc[index_n,'Sequence'],n]
    return df_new

In [10]:
def get_motif_df(df_pos,index_pos,n1,df_neg,index_neg,n2):
    df_sub1 = extract_subdf(df_pos,index_pos,n1)
    df_sub2 = extract_subdf(df_neg,index_neg,n2)
    df_sub = pd.concat([df_sub1,df_sub2])
    df_sub.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
    return df_sub

In [11]:

def encoding(df, bases_list):
    data = np.stack(df.loc[:, 'Sequence'].astype('str').apply(list).values, axis=0)
    e = LabelEncoder()
    e.fit(bases_list)
    encoded = e.transform(data.reshape(-1)).reshape(-1, len(data[0]))
    return encoded

In [12]:
def pca_pep(df, encoded, n=2):
    pca = PCA(n_components=n)
    principalComponents = pca.fit_transform(encoded)
    row, col = principalComponents.shape
    for i in range(col):
        df['pca_{}'.format(i+1)] = principalComponents[:, i]
    return principalComponents, df

In [13]:
def t_sne_pep(df, encoded, n=2):
    t_sne = TSNE(n_components=n, verbose=1, perplexity=50, n_iter=1000, learning_rate=200)
    t_distributed = t_sne.fit_transform(encoded)
    row, col = t_distributed.shape
    for i in range(col):
        df['t_SNE_{}'.format(i+1)] = t_distributed[:, i]
    return t_distributed, df

In [14]:
def mark_origial_seq(df,orig_seq,class_n,normal_size,mark_size):
    df['marker_size'] = 10
    for i in range(len(df)):
        if df.at[i,'Sequence'] == orig_seq:
            df.at[i,'Class']=class_n           
            df.at[i,'marker_size'] = 20
    return df

## Get the PCA and T_SNE vectors of the whole dataset

In [33]:
df_pos = pd.read_csv('Exp6R_n.csv')
df_neg = pd.read_csv('Ctrl6R_n.csv')
df = pd.concat([df_pos,df_neg])
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
bases_list = ['A','T','G','C']
encoded = encoding(df, bases_list)
principalComponents, df_pca = pca_pep(df, encoded, n=2)
#t_distributed, df_sne =  t_sne_pep(df_pca, encoded, n=2)

In [34]:
#load the t-SNE vectors
df_sne = pd.read_csv('t-SNE.csv')
df_sne_top500 = pd.concat([df_sne.head(500),df_sne.tail(500)])
df_pca_top500 = pd.concat([df_pca.head(500),df_pca.tail(500)])
df_sne_top500

,Sequence,Class,pca_1,pca_2,t_SNE_1,t_SNE_2
0,AACACACCACAGACTCTG,1,-0.901205,-0.282037,52.097874,40.674603
1,ACACACCATCAGACGCCG,1,-0.687253,-1.058317,15.336756,50.112247
2,AGCAGCACACGACACACT,1,-0.478427,-1.867341,84.321080,31.045856
3,ACGCCAACACATTCCGCT,1,0.217218,-2.039845,-16.547987,54.125233
4,AACACACACAGCCGTCCG,1,-0.507959,-0.777595,-103.131370,22.906958
...,...,...,...,...,...,...
790301,AGGACTTCCGACGACCTT,0,-0.335315,-1.267226,-36.162360,-67.198740
790302,ACGAACGACACCCCACAG,0,-0.917067,-1.258389,16.644915,24.550121
790303,CCCCGCCTCGCTCCACTA,0,-1.348053,1.777040,45.828903,-34.734447
790304,TCAGGACACCGCCGCAAA,0,-0.142939,2.111133,-54.361416,19.555994


## Choose a motif and get the index of it's occurrence in sequences, change the Class of these sequences

In [52]:
# get overlap motifs from high dff sequences, if run this one, need to comment next cell
bases_list = list('ACGT')
df1 = pd.read_csv('highDff.csv')
df = read_csv('18_nt_length')
df_gap = read_csv('18_nt_gap_length')
dict1 = search_candidates(df1,df)
#dict2 = search_candidates(df1,df_gap)
for x, y in dict1.items():
    print(x,len(y))

ACCCACACC 2
CCCACACCA 2
ACCAGACG 2
AACCAGAC 2
CCAGACGT 2
CAGACGTC 2
CCCACACC 2
ACCCACAC 2
ACACCAAC 2
CCACACCA 2
CACCAACC 2
CACACCAA 2
ACCAACCA 2
GACCCACA 2
ACCACCAA 2
AGCCCTTC 2
CACCACCA 2
CCACCAAC 2
TCACCACC 2
CACCAACT 2
CCCTTCAC 2
CACAAGAC 2
CCAACACC 2
GACCCAAA 2
GCCAACAC 2
CAACACCT 2


In [ ]:
#get top 5 motifs and corresponding indexes from the motif files, if run this cell, need to comment the last cell 
# def read_csv2(pre):
#     path = os.getcwd()
#     #prefix ='18_nt_gap_length'
#     prefix = pre
#     csv_files = glob.glob(os.path.join(path, "{}*".format(prefix)))
#     list_df = []
#     for f in csv_files:
#         df = pd.read_csv(f)
#         df_n = df[:5]
#         list_df.append(df_n)
#     df_c = pd.concat(list_df, ignore_index=True)
#     dict1 = {}
#     for i in range(len(df_c)):
#         dict1[df_c.iloc[i]['motifs']]=[df_c.iloc[i]['index_pos'],df_c.iloc[i]['index_neg']]
#     return dict1       
# dict1 = read_csv2('18_nt_gap_length')
# for x, y in dict1.items():
#     print(x,len(y))

In [56]:
#this cell is not required for plot
#choose one motif from the list above and check which sequence it is in
# motif = 'ACCCACACC'
# x = motif.replace('*','[ATGC]')
# y = motif.replace('*','_')
# for i in range(len(df1)):
#     if re.search(x,df1.iloc[i]['Sequence']):
#         print(x, df1.iloc[i]['Sequence'],df1.iloc[i]['dff'])
#         orig_seq = df1.iloc[i]['Sequence']
#         dff = df1.iloc[i]['dff']



In [57]:
#replace the motif below and get the dataframe of the class 2 and class 3

#motif = 'CACCGATCCT*'
motif = 'ACCCACACC'
index_pos,index_neg = get_motif_index(dict1,motif)
df_v1 = change_class(df_sne,index_pos,2)
index_neg_n = [x + len(df_pos) for x in index_neg]
df_v2 = change_class(df_v1,index_neg_n,3)
df_v3 = df_v2.loc[df_v2['Class'].isin([2,3])]
df_v3


,Sequence,Class,pca_1,pca_2,t_SNE_1,t_SNE_2
113,GACCCACACCAACCAGTG,2,-0.315658,0.850480,1.329096,-29.145927
511,TACCCACACCTCTCCACT,2,1.965829,0.659717,-38.069790,49.438774
521,AACCCACACCACACAGCT,2,-0.483145,-1.554646,26.078812,22.262127
684,AACCCACACCATCACTCG,2,-0.358454,-1.252583,29.157280,3.953075
750,AACCCACACCGACTAGTG,2,-1.299534,-0.041199,74.939370,14.220621
...,...,...,...,...,...,...
784816,GGCATACCCACACCATCT,3,0.791038,-0.847400,-35.942080,7.856315
787998,GACCCACACCATCAGACG,3,0.710397,0.362255,-51.009670,-18.275375
789367,ATTGACCCACACCATGAT,3,0.271091,-2.440686,35.066025,58.245865
789565,ACGCACCCACACCCAACT,3,-0.645779,-1.477957,20.678520,22.695400


In [42]:
# If plot for one dataframe (eg: df_pca_top500, df_sne_top500 or df_v3), comment the second line and the 
#first ax.scatter
# If plot df_v3 against the top 500 background sequence, keep it like this


#def sav_fig(df1, title, x_label, y_label, save_name,  color_column):
def sav_fig(df, df1, title, x_label, y_label, save_name,  color_column):
    
    plt.rcParams.update({'font.size': 45})
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_axes((0.1,0.1,0.5,0.8))
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['top'].set_visible(False)
    ax.set_xlabel('\n' + x_label)
    ax.set_ylabel(y_label + '\n')
    
    labels_dict = {0: '#69BFA0',1: '#FF7F00', 2: '#0000FF', 3:'#FF0000'}
    
   
    ax.scatter(df[x_label], df[y_label],
               edgecolor='none', alpha=0.5,
               c=df[color_column].map(labels_dict), s= 200)
    
    ax.scatter(df1[x_label], df1[y_label],
               edgecolor='none', alpha=0.5,
               c=df1[color_column].map(labels_dict), s= 200)
    

    handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=v, label=k, markersize=15, markeredgecolor = 'none') for k, v in
               labels_dict.items()]
               
    ax.legend(title='Class', handles=handles, bbox_to_anchor=(1.3, 1), loc='upper right', frameon=False, prop={'size': 45},title_fontsize=40)
    
   # fig.suptitle(title, )
    plt.title(title,x=0.43, y=1.1,fontsize= 45)
    #ax.set_title(title)
    ax.set_xlabel( x_label,fontsize= 45)
    ax.set_ylabel(y_label, fontsize= 45)
    plt.savefig(save_name, bbox_inches='tight')
    plt.figure().clear()
    plt.close()
    plt.cla()
    plt.clf()

In [43]:
#sav_fig(df_top500, df_v3,  '18_nt_TSNE_top500_{}_plot'.format(motif), 't_SNE_1', 't_SNE_2', '18_nt_TSNE_top500_{}_v2.png'.format(motif), 'Class')
sav_fig(df_top500,df_v3, '18_nt_PCA_top500_{}_plot'.format(motif), 'pca_1', 'pca_2', '18_nt_PCA_top500_{}.png'.format(motif), 'Class')

<Figure size 1440x720 with 0 Axes>

In [ ]:
# t_distributed_3d, df_sne_3d =  t_sne_pep(df_sub, encoded, n=3)
# principalComponents_3d, df_pca_3d = pca_pep(df_sub, encoded, n=3)
#plot 3d pca
# fig = px.scatter_3d(df_pca_3d, x='pca_1', y='pca_2', z='pca_3',size_max=10,opacity=0.9,size = 'marker_size',
#               color='Class',title="motif: {}, Sequence: {},dff: {}".format(motif,orig_seq,dff))
# fig.show()
# fig.write_html("{}_pca.html".format(y))
#plot 3d tsne
# fig = px.scatter_3d(df_sne_3d, x='t_SNE_1', y='t_SNE_2', z='t_SNE_3',size_max=10,opacity=0.9,size = 'marker_size',
#               color='Class',title="motif: {}, Sequence: {},dff: {}".format(motif,orig_seq,dff))
# fig.show()
# fig.write_html("{}_sne.html".format(y))

In [ ]:
#X = df_sne[['t_SNE_1','t_SNE_2']]
#X = encoded_sub
# neighbors = NearestNeighbors(n_neighbors=4)
# neighbors_fit = neighbors.fit(X)
# distances, indices = neighbors_fit.kneighbors(X)

# distances, indices = neighbors_fit.kneighbors(X)
# distances = np.sort(distances, axis=0)
# distances = distances[:,1]

# df_eps = pd.DataFrame(columns=['distance'])
# df_eps['distance'] = distances
# df_eps['index'] = list(range(len(df_eps)))
# fig = px.line(df_eps, x="index", y="distance", title='To determine eps for dbscan')
# fig.update_layout(height=800)
# fig.show()
#plt.savefig('eps_determin.png')


# nearest_neighbors = NearestNeighbors(n_neighbors=4)
# neighbors = nearest_neighbors.fit(X)

# distances, indices = neighbors.kneighbors(X)
# distances = np.sort(distances[:,3], axis=0)

# fig = plt.figure(figsize=(8, 8))
# plt.plot(distances)
# plt.xlabel("Points")
# plt.ylabel("Distance")

In [ ]:
# def dbscan(df,Eps,Min_samples):
#     X = df
#     X = StandardScaler().fit_transform(X)
#     db = DBSCAN(eps=Eps, min_samples=Min_samples).fit(X)
#     labels = db.labels_
#     df_dbscan = df_sne.copy()
#     df_dbscan['Labels'] = labels 
#     core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
#     core_samples_mask[db.core_sample_indices_] = True
#     df_dbscan.to_csv('dbscan_{}.csv'.format(y))
#     unique_labels = set(labels)
#     return df_dbscan,unique_labels
    

In [ ]:
# df_dbscan,unique_labels = dbscan(encoded,1.5,4)
# df_dbscan
# unique_labels

In [ ]:
# fig = px.scatter(df_v3, x="t_SNE_1", y="t_SNE_2", hover_name="Class", color = "Class", size_max=60)
# fig.update_layout(height=800)
# fig.show()

In [ ]:
# X = encoded
# X = StandardScaler().fit_transform(X)
# db = DBSCAN(eps=0.5, min_samples=4).fit(X)
# labels = db.labels_
# df_dbscan = df_sne.copy()
# df_dbscan['Labels'] = labels 
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# df_dbscan.to_csv('dbscan_{}.csv'.format(y))
# unique_labels = set(labels)

# fig = plt.figure(figsize = (8,8))
# colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
# for k, col in zip(labels, colors):
#     if k == -1:
#         # Black used for noise.
#         col = [0, 0, 0, 1]
#     class_member_mask = labels == k

#     xy = X[class_member_mask & core_samples_mask]
#     plt.plot(xy[:, 0],xy[:, 1], "o",markerfacecolor=tuple(col),markeredgecolor="k",markersize=7)

#plt.title("Estimated number of clusters: %d" % n_clusters_)
#plt.show()